In [27]:
# PDF 파일을 \n\n 단위로 우선 분할, 길이 초과시 토큰 단위로 300토큰씩 30토큰 오버랩 분할
import pymupdf
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
ISPDF = False


def split_by_paragraph(text):
  return [p.strip() for p in text.split("\n\n") if p.strip()]


def chunk_tokens(tokens, chunk_size=300, overlap=30, title=None):
  chunks = []
  start = 0
  while start < len(tokens):
    end = min(start + chunk_size, len(tokens))
    chunk_tokens = tokens[start:end]
    chunk_text = tokenizer.decode(chunk_tokens)
    if title:
      chunk_text = f"{title}\n{chunk_text}"
    chunks.append(chunk_text)
    if end == len(tokens):
      break
    start += chunk_size - overlap
  return chunks


def smart_chunk_text(paragraphs, chunk_size=300, overlap=30):
  result_chunks = []
  for para in paragraphs:
    tokens = tokenizer.encode(para, add_special_tokens=False)
    if len(tokens) <= chunk_size:
      result_chunks.append(para)
    else:
      title = None
      if para.startswith("#"):
        title = para.split("\n")[0]
      result_chunks.extend(chunk_tokens(tokens, chunk_size, overlap, title))
  return result_chunks

## PDF


In [ ]:
text = ""
# PDF 파일에서 텍스트 추출
result = pymupdf.open("example.pdf")
for page in result:
  text += page.get_text()
chunked_subjects = smart_chunk_text(
  split_by_paragraph(text), chunk_size=300, overlap=30)
print(f"총 청크 개수: {len(chunked_subjects)}")
print(chunked_subjects[0])  # 첫 청크 일부 출력

Token indices sequence length is longer than the specified maximum sequence length for this model (9224 > 512). Running this sequence through the model will result in indexing errors


총 청크 개수: 35
1주차 1열 운영체제 개요 2열 3/13 Status 완료 문제집 (중간고사 대비) 4장_CPU 스케줄링.pdf 3장_프로세스와 스레드.pdf 1장_운영체제의 개요.pdf 평가 관련 안내 1학기 : 중간고사(40%) + 노션 정리(30%) + 논술(30%) 2학기 : 미정 1~2주차 : 가볍게 읽고, 내용 이해하기 단, 커널의 모드와 종류(모놀로틱, 마이크로)에 대해서는 정리하기 운영체제 개요 1주차 1 1. 운영체제 1. 컴퓨터 자원을 효율적으로 관리하고 운영하는 시스템 소프트웨어 2. 사용자가 편리하게 컴퓨터를 활용할 수 있는 환경을 제공 3. 목표 : 범용성, 확장성, 적응성, 처리 능력 향상, 응답 시간 단축, 사용의 용이성, 가동성 향 상 1.1. 운영체제의 주요 목적 사용의 용이성 하드웨어와 정보를 효율적으로 관리하여 컴퓨터를 편리하게 사용할 수 있도록 지원 시스템 성능 향상 처리 능력 (throughput): 일정 시간 동안 컴퓨터가 처리하는 작업량 응답 시간 (turnaround time): 사용자가 요청한 작업이 완료되기까지의 시간 사용의 용이성 (availability): 시스템 자원의 신속한 제공 여부 신뢰도 (reliability): 시스템의 정확한 동작 정도 1.2. 운영체제의 기능 자원 관리와 시스템 관리로 구분 


## MarkDown


In [28]:
import re

text = ""
with open("example.md", "r", encoding="utf-8") as f:
  text = f.read()

# Split by markdown headings, keeping the heading marker
pattern = r'(^#+ .*)'
splits = re.split(pattern, text, flags=re.MULTILINE)
subjects = []

# Reconstruct sections: heading + content
for i in range(1, len(splits), 2):
  heading = splits[i].strip()
  content = splits[i + 1].strip() if i + 1 < len(splits) else ""
  subjects.append(f"{heading}\n{content}")

print(f"총 단원 개수: {len(subjects)}")
print(subjects[0:5])

총 단원 개수: 51
['# 운영체제 개요\n', '## 1. 운영체제\n1. 컴퓨터 자원을 효율적으로 관리하고 운영하는 시스템 소프트웨어\n2. 사용자가 편리하게 컴퓨터를 활용할 수 있는 환경을 제공\n3. 목표 : 범용성, 확장성, 적응성, 처리 능력 향상, 응답 시간 단축, 사용의 용이성, 가동성 향상', '### 1.1. 운영체제의 주요 목적\n- **사용의 용이성**\n    - 하드웨어와 정보를 효율적으로 관리하여 컴퓨터를 편리하게 사용할 수 있도록 지원\n- **시스템 성능 향상**\n    - **처리 능력 (throughput)**: 일정 시간 동안 컴퓨터가 처리하는 작업량\n    - **응답 시간 (turnaround time)**: 사용자가 요청한 작업이 완료되기까지의 시간\n    - **사용의 용이성 (availability)**: 시스템 자원의 신속한 제공 여부\n    - **신뢰도 (reliability)**: 시스템의 정확한 동작 정도', '### 1.2. 운영체제의 기능\n**자원 관리**와 **시스템 관리**로 구분\n\n- **자원 관리(중요)**: **메모리 관리, 프로세스 관리, 주변 장치 관리, 파일(데이터) 관리**\n- **시스템 관리**: 시스템 보호(사용자 권한 관리), 네트워킹(통신), 명령 인터프리터 제공', '### 1.3. 운영체제의 기능 분류\n- 감시 프로그램 : 작업 관리, 자원 할당과 회수 등 시스템 전반을 감독하고 제어함\n- 작업 제어 프로그램 : 작업의 순서를 정하고 작업 흐름을 통제함\n- 데이터 관리 프로그램 : 파일, 데이터의 표준적 관리 및 처리를 담당함\n\n- 참고 : 운영체제의 역할인 것, 역할이 아닌 것\n- 역할인 것\n    - 사용자 인터페이스 제공\n    - 주변 장치 관리\n    - 자원 분배와 효율적 관리\n    - 신뢰성 향상 및 오류 처리\n- 역할이 아닌 것\n    - 원시 프로그램을 기계어로 번역하는 기능 (컴파일러의 역할)\n    - 목적 프로그램의

In [ ]:
import requests
import dotenv
import os

dotenv.load_dotenv()
DifyChapterToken = os.getenv("DifyChapterToken")

titles = []

for i in subjects:
  if i.startswith("#"):
    titles.append(i.split("\n")[0])
res = requests.post(
  "https://dify.obtuse.kr/v1/completion-messages",
  headers={
    'Authorization': f'Bearer {DifyChapterToken}',
    'Content-Type': 'application/json'
  },
  json={
    "inputs": {
      "query": "\n".join(titles),
    },
    "response_mode": "blocking",
    "user": "asdf-1234"
  }
)

In [36]:
res.json()

{'event': 'message',
 'task_id': 'c31b71be-8f5c-473c-b9b3-ee9f18555dff',
 'id': '8bd00a21-f9db-4c29-9ba5-1db4d9ccd6e0',
 'message_id': '8bd00a21-f9db-4c29-9ba5-1db4d9ccd6e0',
 'mode': 'completion',
 'answer': '{"chapters": [{"title": "운영체제 개요", "sections": ["운영체제의 정의와 목표", "운영체제의 주요 기능", "운영체제의 기능 분류", "운영체제 구성요소와 역할", "커널과 쉘의 역할"]}, {"title": "운영체제 연산", "sections": ["트랩과 시스템 호출", "사용자 모드와 커널 모드"]}, {"title": "운영체제 구조", "sections": ["간단한 구조", "계층적 접근", "마이크로커널", "모놀리식 커널"]}, {"title": "운영체제 발달 과정", "sections": ["일괄처리 시스템", "다중 프로그래밍 시스템", "시분할 시스템", "다중 처리 시스템", "실시간 처리 시스템", "다중 모드 처리 시스템", "분산 처리 시스템"]}, {"title": "프로세스", "sections": ["프로세스 개요", "프로그램과 프로세스의 차이", "프로세스의 메모리 구조", "프로세스 제어 블록(PCB)과 역할", "프로세스 상태와 상태 전이"]}, {"title": "프로세스와 스레드", "sections": ["프로세스와 스레드 비교", "싱글 스레드와 멀티 스레드", "커널 스레드와 사용자 스레드"]}, {"title": "프로세스 스케줄링", "sections": ["스케줄링 개요", "스케줄링의 종류(기능별, 방법별)", "선입선출 스케줄링(FCFS)", "최단 작업 우선 스케줄링(SJF)", "최단 잔여 시간 우선 스케줄링(SRT)", "HRN 스케줄링", "우선순위 스케줄링", "라운드로빈 스케줄링(RR)"

In [29]:
# 각 단원에 대해 청크 분할
chunked_subjects = smart_chunk_text(subjects, chunk_size=300, overlap=30)
chunked_subjects

Token indices sequence length is longer than the specified maximum sequence length for this model (897 > 512). Running this sequence through the model will result in indexing errors


['# 운영체제 개요\n',
 '## 1. 운영체제\n1. 컴퓨터 자원을 효율적으로 관리하고 운영하는 시스템 소프트웨어\n2. 사용자가 편리하게 컴퓨터를 활용할 수 있는 환경을 제공\n3. 목표 : 범용성, 확장성, 적응성, 처리 능력 향상, 응답 시간 단축, 사용의 용이성, 가동성 향상',
 '### 1.1. 운영체제의 주요 목적\n- **사용의 용이성**\n    - 하드웨어와 정보를 효율적으로 관리하여 컴퓨터를 편리하게 사용할 수 있도록 지원\n- **시스템 성능 향상**\n    - **처리 능력 (throughput)**: 일정 시간 동안 컴퓨터가 처리하는 작업량\n    - **응답 시간 (turnaround time)**: 사용자가 요청한 작업이 완료되기까지의 시간\n    - **사용의 용이성 (availability)**: 시스템 자원의 신속한 제공 여부\n    - **신뢰도 (reliability)**: 시스템의 정확한 동작 정도',
 '### 1.2. 운영체제의 기능\n**자원 관리**와 **시스템 관리**로 구분\n\n- **자원 관리(중요)**: **메모리 관리, 프로세스 관리, 주변 장치 관리, 파일(데이터) 관리**\n- **시스템 관리**: 시스템 보호(사용자 권한 관리), 네트워킹(통신), 명령 인터프리터 제공',
 '### 1.3. 운영체제의 기능 분류\n- 감시 프로그램 : 작업 관리, 자원 할당과 회수 등 시스템 전반을 감독하고 제어함\n- 작업 제어 프로그램 : 작업의 순서를 정하고 작업 흐름을 통제함\n- 데이터 관리 프로그램 : 파일, 데이터의 표준적 관리 및 처리를 담당함\n\n- 참고 : 운영체제의 역할인 것, 역할이 아닌 것\n- 역할인 것\n    - 사용자 인터페이스 제공\n    - 주변 장치 관리\n    - 자원 분배와 효율적 관리\n    - 신뢰성 향상 및 오류 처리\n- 역할이 아닌 것\n    - 원시 프로그램을 기계어로 번역하는 기능 (컴파일러의 역할)\n    - 목적 프로그램의 생성 및 연결

In [30]:
# 분할된 청크를 chroma db에 embedding function과 함께 등록
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.Client()
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large"
)

collection = chroma_client.create_collection(
    name="pdf_chunks_collection", embedding_function=sentence_transformer_ef)

chunk_ids = [f"chunk_{i}" for i in range(len(chunked_subjects))]
collection.add(
    documents=chunked_subjects,
    ids=chunk_ids
)
print(f"{len(chunked_subjects)}개의 청크가 chroma db에 등록되었습니다.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


57개의 청크가 chroma db에 등록되었습니다.


In [37]:
# 검색 쿼리 예시 및 RAG에 활용할 수 있는 검색 코드
query = "운영 프로그램의 기능 분류 중 감시 프로그램이란?"
results = collection.query(
    query_texts=[query],
    n_results=3
)
for i, result in enumerate(zip(results['documents'][0], results['distances'][0])):
  print(f"Top {i + 1} 결과:\n{result[0]}\n거리:{result[1]}\n---")

Top 1 결과:
### 1.3. 운영체제의 기능 분류
- 감시 프로그램 : 작업 관리, 자원 할당과 회수 등 시스템 전반을 감독하고 제어함
- 작업 제어 프로그램 : 작업의 순서를 정하고 작업 흐름을 통제함
- 데이터 관리 프로그램 : 파일, 데이터의 표준적 관리 및 처리를 담당함

- 참고 : 운영체제의 역할인 것, 역할이 아닌 것
- 역할인 것
    - 사용자 인터페이스 제공
    - 주변 장치 관리
    - 자원 분배와 효율적 관리
    - 신뢰성 향상 및 오류 처리
- 역할이 아닌 것
    - 원시 프로그램을 기계어로 번역하는 기능 (컴파일러의 역할)
    - 목적 프로그램의 생성 및 연결(링커의 역할)
    - 데이터 압축 및 복원(유틸리티 프로그램 역할)
거리:0.26685482263565063
---
Top 2 결과:
### 1.2. 운영체제의 기능
**자원 관리**와 **시스템 관리**로 구분

- **자원 관리(중요)**: **메모리 관리, 프로세스 관리, 주변 장치 관리, 파일(데이터) 관리**
- **시스템 관리**: 시스템 보호(사용자 권한 관리), 네트워킹(통신), 명령 인터프리터 제공
거리:0.3587704598903656
---
Top 3 결과:
### 2.2. 사용자 모드와 커널 모드
- 참고 : 커널과 쉘 - 커널 - 운영체제의 중심부로 하드웨어를 직접 관리하고 제어하는 핵심적 역할을 수행 - 주된 역할: - 프로세스 관리(생성, 종료, 스케줄링 등) - 메모리 관리(할당, 회수, 가상 메모리 등) - 파일 시스템 관리(파일 입출력, 파일 접근 관리 등) - 입출력 장치 관리 및 하드웨어 자원 관리 - 커널은 운영체제의 가장 내부에서 작동하여 응용 프로그램과 하드웨어 사이의 중재자 역할을 수행 - 쉘 - 쉘은 사용자가 운영체제와 상호작용할 수 있도록 제공하는 **사용자 인터페이스(UI)** 프로그램 - 주된 역할: - 사용자 명령어를 해석하여 운영체제의 커널에 전달함. - 사용자의 명령을 실행하고 결과를 다시 사용자